In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# The QLattice

I previous did a notebook on this dataset with a very simple approach, letting the QLattice do all the work with no EDA or data prep at all. You can see it here: https://www.kaggle.com/karinbondgaard/ml-your-grandmother-can-do. 
In this notebook I'll still use a very simple approach letting the QLattice do the feature selection for me, but this time I will create some new features from the datetime column to see if the QLattice picks up usable signal from those.
The QLattice can also be used for much more advanced analysis, but you have to find examples of that in other notebooks.

The QLattice is a supervised machine learning tool for symbolic regression developed by Abzu. It is inspired by Richard Feynman's path integral formulation. That's why the python module to use it is called Feyn, and the Q in QLattice is for Quantum.

Abzu provides free QLattices for non-commercial use to anyone. These free community QLattices gets allocated for you automatically if you use Feyn without an active subscription, as we will do in this notebook. Read more about how it works here: https://docs.abzu.ai/docs/guides/getting_started/community.html

In [ ]:
# install the feyn module
!pip install feyn

In [ ]:
import feyn
import sklearn.model_selection
import seaborn as sbn

In [ ]:
# loading the dataset
df_polution = pd.read_csv("../input/tabular-playground-series-jul-2021/train.csv")
df_polution["date_time"] = pd.to_datetime(df_polution["date_time"], format="%Y-%m-%d %H:%M:%S")

# Calculating new features 

I will calculate new features based on the date_time:
* name of month
* name of day
* season
* is weekend
* time of day

The idea of also calculating a SMC feature was taken from this [notebook](https://www.kaggle.com/maximkazantsev/tps-07-21-eda-lightautoml).

In [ ]:
def calculate_new_features(df):
    
    df["month"] = df["date_time"].dt.month_name()
    df["day_of_week"] = df['date_time'].dt.day_name()
        
    
    df.loc[df["date_time"].dt.month.isin([12,1,2]), "season"] = "winter"
    df.loc[df["date_time"].dt.month.isin([3,4,5]), "season"] = "spring"
    df.loc[df["date_time"].dt.month.isin([6,7,8]), "season"] = "summer"
    df.loc[df["date_time"].dt.month.isin([9,10,11]), "season"] = "autum"
            
    df["is_weekend"] = (df["date_time"].dt.dayofweek >= 5).astype("int")
    
    
    df.loc[df["date_time"].dt.hour.isin(np.arange(8, 17, 1)), "time_of_day"] = "day"
    df.loc[df["date_time"].dt.hour.isin(np.arange(17, 24, 1)), "time_of_day"] = "evening"
    df.loc[df["date_time"].dt.hour.isin(np.arange(0, 8, 1)), "time_of_day"] = "night" 
    
    df['SMC'] = (df['absolute_humidity'] * 100) / df['relative_humidity']
        
    return df

In [ ]:
df_polution = calculate_new_features(df_polution)
df_polution = df_polution = df_polution.drop(["date_time"], axis=1)
stypes = {"season": "cat",
          "time_of_day": "cat",
          "month": "cat",
          "day_of_week": "cat"
         }

In [ ]:
# preparing a dataset for predicting each variable
df_target_carbon_monoxide = df_polution
df_target_carbon_monoxide = df_target_carbon_monoxide.drop(["target_benzene", 
                                                            "target_nitrogen_oxides"], 
                                                           axis=1)

df_target_benzene = df_polution
df_target_benzene = df_target_benzene.drop(["target_carbon_monoxide", 
                                            "target_nitrogen_oxides"], 
                                           axis=1)

df_target_nitrogen_oxides = df_polution
df_target_nitrogen_oxides = df_target_nitrogen_oxides.drop(["target_carbon_monoxide", 
                                                            "target_benzene"], 
                                                           axis=1)

In [ ]:
# splitting the datasets
df_train_target_carbon_monoxide, df_test_target_carbon_monoxide = sklearn.model_selection.train_test_split(df_target_carbon_monoxide, 
                                                                                                           train_size=.80, 
                                                                                                           random_state=1)
df_train_target_benzene, df_test_target_benzene = sklearn.model_selection.train_test_split(df_target_benzene, 
                                                                                           train_size=.80, 
                                                                                           random_state=1)
df_train_target_nitrogen_oxides, df_test_target_nitrogen_oxides = sklearn.model_selection.train_test_split(df_target_nitrogen_oxides, 
                                                                                                           train_size=.80, 
                                                                                                           random_state=1)

In [ ]:
# connect to a qlattice
ql = feyn.connect_qlattice()

In [ ]:
# training model for predicting carbon_monoxide
models = ql.auto_run(df_train_target_carbon_monoxide,
                     output_name="target_carbon_monoxide",
                     stypes = stypes)

In [ ]:
# plotting the best model the qlattice found
best_model = models[0]
best_model.plot(df_train_target_carbon_monoxide, df_test_target_carbon_monoxide)

In [ ]:
# loading test data
df_polution_test = pd.read_csv("../input/tabular-playground-series-jul-2021/test.csv")
df_polution_test["date_time"] = pd.to_datetime(df_polution_test["date_time"], format="%Y-%m-%d %H:%M:%S")
calculate_new_features(df_polution_test) 

In [ ]:
# preparring for building submission df
pred_target_carbon_monoxide = best_model.predict(df_polution_test)

In [ ]:
# training model for predicting benzene
ql.reset()
models = ql.auto_run(df_train_target_benzene, 
                     output_name="target_benzene",
                     stypes = stypes)

In [ ]:
# plotting the best model the qlattice found
best_model = models[0]
best_model.plot(df_train_target_benzene, df_test_target_benzene)

In [ ]:
# preparring for building submission df
pred_target_benzene = best_model.predict(df_polution_test)

In [ ]:
# training model for predicting nitrogen_oxides
ql.reset()
models = ql.auto_run(df_train_target_nitrogen_oxides, 
                     output_name="target_nitrogen_oxides",
                     stypes = stypes)

In [ ]:
# plotting the best model the qlattice found
best_model = models[0]
best_model.plot(df_train_target_nitrogen_oxides, df_test_target_nitrogen_oxides)

# Conclusion

The new features did indeed improve my predictions compared to my first notebook on this dataset!

In [ ]:
# preparring for building submission df
pred_target_nitrogen_oxides = best_model.predict(df_polution_test)

In [ ]:
# building submission csv
df_my_submission = pd.DataFrame(columns=["date_time", "target_carbon_monoxide", "target_benzene", "target_nitrogen_oxides"])
df_my_submission = df_my_submission.astype({"date_time": object, "target_carbon_monoxide": float, "target_benzene": float, "target_nitrogen_oxides": float})

for i in range(len(df_polution_test)):
    new_row = {"date_time":df_polution_test["date_time"].values[i], 
               "target_carbon_monoxide": pred_target_carbon_monoxide[i],
               "target_benzene": pred_target_benzene[i],
               "target_nitrogen_oxides": pred_target_nitrogen_oxides[i]}    
    df_my_submission = df_my_submission.append(new_row, ignore_index=True)  

df_my_submission.to_csv('submission.csv', index=False)